## 환경설정

1. http://braincrew2.iptime.org:8001 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 기입하시는 것을 권장 드려요. 예. 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.


In [1]:
project = "BBCTEXT"  # 수정하지 마세요
username = "longtailedtit12@gmail.com"  # 이메일아이디 (예시. abc@hello.com)
password = "qwer1234"  # 비밀번호

아래의 코드를 순서대로 실행해 주세요.


In [2]:
import os
import requests

if not os.path.exists("competition.py"):
    url = "https://link.teddynote.com/COMPT"
    file_name = "competition.py"
    response = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(response.content)

In [3]:
import competition

# 파일 다운로드
competition.download_competition_files(
    f"https://link.teddynote.com/{project}", use_competition_url=False
)

100%|██████████| 1.80M/1.80M [00:00<00:00, 25.3MiB/s]


## 데이터 로드


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os

# Data 경로 설정
DATA_DIR = "data"

# 경고 무시
warnings.filterwarnings("ignore")

SEED = 123

train = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

In [5]:
# train 데이터셋 확인
train.head()

,content,label
0,officials respond in court row australian tenn...,sport
1,slow start to speedy net services faster broad...,tech
2,amnesty chief laments war failure the lack of ...,politics
3,dal maso in to replace bergamasco david dal ma...,sport
4,technology gets the creative bug the hi-tech a...,tech


In [6]:
# test 데이터셋 확인
test.head()

,content
0,child access laws shake-up parents who refuse ...
1,fry set for role in hitchhiker s actor stephen...
2,palestinian economy in decline despite a short...
3,japanese banking battle at an end japan s sumi...
4,manufacturing recovery slowing uk manufactur...


## HugginFace

- Model & Tokenizer 다운로드
- 링크: https://huggingface.co/abhishek/autonlp-bbc-news-classification-37229289


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(
    "abhishek/autonlp-bbc-news-classification-37229289"
)

model = AutoModelForSequenceClassification.from_pretrained(
    "abhishek/autonlp-bbc-news-classification-37229289"
)

tokenizer_config.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

## 토큰화된 결과 확인

- `input_ids`, `token_type_ids`, `attention_mask`로 구성되어 있음
- `input_ids` 는 변환된 id 토큰을 의미
- `token_type_ids` 는 2개 이상의 문장이 이어지는 문장인지 아닌지를 판단하는 task를 수행


In [8]:
tokenized = tokenizer(train["content"].iloc[0], padding=True, truncation=True)
tokenized

{'input_ids': [101, 4584, 6869, 1999, 2457, 5216, 2827, 5093, 2327, 2880, 2038, 8047, 1996, 2827, 2330, 5434, 2012, 4940, 2380, 2044, 6256, 1997, 1996, 2652, 3302, 2011, 2222, 3240, 2669, 19482, 1012, 19482, 2056, 2002, 2018, 2018, 1037, 9535, 3993, 1997, 2667, 2000, 13984, 2068, 2000, 2191, 1996, 3302, 5514, 2021, 5093, 2660, 1055, 14915, 25513, 5837, 2010, 7928, 1012, 2057, 2106, 3198, 2005, 2009, 2000, 2022, 2081, 5514, 2084, 2197, 2095, 1998, 2000, 2026, 3716, 2009, 2003, 5514, 2002, 2056, 1012, 2057, 2985, 1037, 6937, 3815, 1997, 2769, 2006, 12719, 2000, 2191, 2009, 5514, 1012, 19482, 2040, 2038, 2025, 2288, 2627, 1996, 2959, 2461, 2012, 4940, 1999, 2698, 4740, 2018, 3041, 2056, 1996, 3302, 2001, 2025, 11007, 1997, 2107, 1037, 8919, 2977, 1012, 2021, 2002, 2056, 2002, 2052, 2377, 2006, 3649, 3302, 2002, 2018, 2000, 1999, 2344, 2000, 2031, 1037, 3382, 1997, 3045, 1012, 1996, 2977, 4627, 2006, 2459, 2254, 1998, 25513, 2056, 1996, 3302, 2018, 2042, 3177, 2098, 2039, 2144, 2197, 2095,

In [9]:
# input_ids 10개만 출력
input_ids = tokenized["input_ids"]
input_ids[:10]

[101, 4584, 6869, 1999, 2457, 5216, 2827, 5093, 2327, 2880]

- 시작 토큰은 `[CLS]`, 끝나는 토큰은 `[SEP]` 토큰이 위치합니다.


In [10]:
# 변환된 결과 확인
# 처음 20개의 토큰
print(tokenizer.convert_ids_to_tokens(input_ids)[:20])

['[CLS]', 'officials', 'respond', 'in', 'court', 'row', 'australian', 'tennis', 'top', 'official', 'has', 'defended', 'the', 'australian', 'open', 'courts', 'at', 'melbourne', 'park', 'after']


In [11]:
# 끝부분의 20개의 토큰
print(tokenizer.convert_ids_to_tokens(input_ids)[-20:])

['be', 'faster', 'than', 'the', 'one', 'll', '##ey', '##ton', 'beat', 'roger', 'federer', 'on', 'in', 'the', 'davis', 'cup', 'in', '2003', '.', '[SEP]']


## Label 맵 생성


In [12]:
# label 맵과 idx_to_label 생성
label_map = {v: i for i, v in enumerate(train["label"].value_counts().keys())}
idx_to_label = {idx: lbl for lbl, idx in label_map.items()}

In [13]:
train["label_num"] = train["label"].map(label_map)

## Dataset 분할


In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    train["content"], train["label_num"], test_size=0.2, random_state=SEED
)

## Batch Tokenization

- 배치(batch) 단위로 묶어서 토큰 처리


![](https://mlspring.files.wordpress.com/2022/12/attention_mask.png?w=640)


In [17]:
# truncation 시 최대길이 확인 - 모델마다 최대길이는 지정되어 있음
tokenizer.model_max_length

512

- `truncation`: `model_max_length` 길이에 맞춰 잘라냄
- `padding`: `model_max_length` 길이보다 짧으면 패딩(padding_index=0)으로 채움


In [18]:
# 배치 단위의 사이즈를 맞춰 일괄 처리
batch_tokenized = tokenizer(
    train["content"].iloc[:10].tolist(), padding=True, truncation=True
)

In [19]:
# (batch_size, model_max_length)
np.array(batch_tokenized["input_ids"]).shape

(10, 512)

## Dataset 생성


In [20]:
from torch.utils.data import DataLoader, Dataset


class CustomDataset(Dataset):
    def __init__(self, texts, labels, is_train=True):
        super().__init__()
        self.texts = texts
        self.is_train = is_train
        if is_train:
            self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        if self.is_train:
            label = self.labels.iloc[idx]
            return text, label
        else:
            return text

In [21]:
# Custom Dataset 생성
train_ds = CustomDataset(
    x_train,
    y_train,
)
valid_ds = CustomDataset(x_test, y_test)
test_ds = CustomDataset(test["content"], None, is_train=False)

In [22]:
# 1개의 데이터 추출
text, label = next(iter(train_ds))
len(text), label

(1499, 0)

## DataLoader 생성


In [23]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

# CUDA 사용 가능 여부 확인
if torch.backends.mps.is_built():
    # mac os mps 지원 체크
    device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
else:
    # cuda 사용 가능한지 체크
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [24]:
def collate_batch(batch, tokenizer, is_train=True):
    if is_train:
        text_list, label_list = [], []

        for text, label in batch:
            text_list.append(text)
            label_list.append(label)

        label_list = torch.tensor(label_list, dtype=torch.int64)

        # padding을 주어 짧은 문장에 대한 길이를 맞춥니다.
        text_tokenized = tokenizer(
            text_list, padding=True, truncation=True, return_tensors="pt"
        )

        return text_tokenized, label_list

    else:
        text_list = []

        for text in batch:
            text_list.append(text)

        # padding을 주어 짧은 문장에 대한 길이를 맞춥니다.
        text_tokenized = tokenizer(
            text_list, padding=True, truncation=True, return_tensors="pt"
        )

        return text_tokenized

In [25]:
train_loader = DataLoader(
    train_ds,
    batch_size=32,
    shuffle=True,
    collate_fn=lambda x: collate_batch(x, tokenizer),
)

validation_loader = DataLoader(
    valid_ds,
    batch_size=32,
    shuffle=False,
    collate_fn=lambda x: collate_batch(x, tokenizer),
)

test_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: collate_batch(x, tokenizer, is_train=False),
)

In [26]:
x, y = next(iter(train_loader))
x = x.to(device)
y = y.to(device)

# x, y의 shape 확인
# (batch_size, seq_length), (batch_size)
x["input_ids"].shape, y.shape

(torch.Size([32, 512]), torch.Size([32]))

## 모델


In [27]:
from tqdm import tqdm  # Progress Bar 출력
import numpy as np
import torch.nn as nn
import torch.optim as optim

# 사전학습 모델의 구조 확인
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [28]:
# 가중치 Freeze
for param in model.parameters():
    param.requires_grad = False

In [29]:
# 모델의 classifier 부분 변경
model.classifier = nn.Sequential(
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Linear(256, 32),
    nn.ReLU(),
    nn.Linear(32, 5),
)

In [30]:
# 변경된 classifier 가중치 업데이트 가능 여부 확인
for param in model.classifier.parameters():
    print(param.requires_grad)

True
True
True
True
True
True


In [31]:
model = model.to(device)

## 손실함수 및 옵티마이저 정의


In [32]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [33]:
from tqdm import tqdm


def fit(model, data_loader, loss_fn, optimizer, device, phase="train"):
    if phase == "train":
        # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
        model.train()
    else:
        # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
        model.eval()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(
        data_loader, leave=False, unit="batch", total=len(data_loader), mininterval=1
    )

    # mini-batch 학습을 시작합니다.
    for txt, lbl in prograss_bar:
        # txt, lbl 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k: v.to(device) for k, v in txt.items()}
        lbl = lbl.to(device)

        optimizer.zero_grad()
        # 누적 Gradient를 초기화 합니다.
        with torch.set_grad_enabled(phase == "train"):
            # Forward Propagation을 진행하여 결과를 얻습니다.
            output = model(**inputs)

            # HuggingFace BERT 모델의 출력은 logits 로 가져옵니다
            output = output.logits

            # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
            loss = loss_fn(output, lbl)

            if phase == "train":
                # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
                loss.backward()

                # 계산된 Gradient를 업데이트 합니다.
                optimizer.step()

        # output 의 뉴런별 확률 값을 sparse vector 로 변환합니다.
        pred = output.argmax(axis=1)

        # 정답 개수를 카운트 합니다.
        corr += (lbl == pred).sum().item()

        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item()

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader), acc

In [34]:
import time

# 최대 Epoch을 지정합니다.
num_epochs = 3

min_loss = np.inf

STATE_DICT_PATH = "BBC-Text-HF-Classification.pth"

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    start = time.time()
    train_loss, train_acc = fit(
        model, train_loader, loss_fn, optimizer, device, phase="train"
    )

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = fit(
        model, validation_loader, loss_fn, optimizer, device, phase="eval"
    )

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(
            f"[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!"
        )
        min_loss = val_loss
        torch.save(model.state_dict(), STATE_DICT_PATH)

    time_elapsed = time.time() - start
    # Epoch 별 결과를 출력합니다.
    print(
        f"[Epoch{epoch+1:02d}] time: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s \t loss: {train_loss:.5f}, acc: {train_acc:.5f} | val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}"
    )

[INFO] val_loss has been improved from inf to 0.38209. Saving Model!
[Epoch01] time: 3m 51s 	 loss: 0.93339, acc: 0.94593 | val_loss: 0.38209, val_acc: 0.98876


[INFO] val_loss has been improved from 0.38209 to 0.10336. Saving Model!
[Epoch02] time: 3m 50s 	 loss: 0.22327, acc: 0.99368 | val_loss: 0.10336, val_acc: 0.98876


[INFO] val_loss has been improved from 0.10336 to 0.06389. Saving Model!
[Epoch03] time: 3m 45s 	 loss: 0.07053, acc: 0.99438 | val_loss: 0.06389, val_acc: 0.98876


## 저장한 가중치 로드


In [35]:
# 모델에 저장한 가중치를 로드합니다.
model.load_state_dict(torch.load(STATE_DICT_PATH))

<All keys matched successfully>

## 최종 검증 손실 및 정확도 출력


In [36]:
# 최종 검증 손실(validation loss)와 검증 정확도(validation accuracy)를 산출합니다.
final_loss, final_acc = fit(
    model, validation_loader, loss_fn, optimizer, device, phase="eval"
)
print(f"\nevaluation loss: {final_loss:.5f}, evaluation accuracy: {final_acc:.5f}")


evaluation loss: 0.06389, evaluation accuracy: 0.98876


## 예측코드


In [37]:
predictions = []
model = model.to(device)
# 검증모드 진입
model.eval()

with torch.no_grad():
    # loss 초기화
    running_loss = 0
    # 정확도 계산
    running_acc = 0
    for x in test_loader:
        x = x.to(device)
        inputs = {k: v.to(device) for k, v in x.items()}
        y_hat = model(**inputs)
        y_hat = y_hat.logits
        label = y_hat.argmax(dim=1).detach().item()
        predictions.append(label)

In [38]:
your_answer = [idx_to_label[p] for p in predictions]
your_answer[:5]

['politics', 'entertainment', 'business', 'business', 'business']

## 결과 제출

- 느리다고 중지 후 다시 평가 코드를 실행하는 경우 제출 과정에서 패널티가 발생할 수 있습니다. (제출 횟수 이슈 발생 가능)
- 제출결과는 [대회페이지](http://braincrew2.iptime.org:8001/competitions/BBCTEXT/)의 `리더보드` 와 `제출` 탭에서 확인할 수 있습니다.


아래 Cell을 실행하여 예측 결과 업데이트


In [39]:
import competition

# 예측 결과 업데이트
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["label"] = your_answer

display(submission)
competition.submit(project, username, password, submission)

,label
0,politics
1,entertainment
2,business
3,business
4,business
...,...
440,business
441,politics
442,entertainment
443,business


아이디:  longtailedtit12@gmail.com
파일명:  submissions/20240228-113352-submission.csv
[제출에 성공하였습니다]
제출 결과: 0.9842696629213483
